In [1]:
import sys
sys.path.append("../..") 
from cryoem.projections import generate_2D_projections
import numpy as np
from mayavi import mlab

In [2]:
PROTEIN = "5j0n"
ANGLE_COVERAGE = [2.0, 1.0, 2.0] 
ANGLE_SHIFT     = [0., 0., 0.]
PROJECTIONS_NUM_SINGLE = 2000
DTYPE = np.float32

In [3]:
generate_2D_projections(input_file_path=f"../../data/{PROTEIN}.mrc", 
                            ProjNber=PROJECTIONS_NUM_SINGLE,
                            AngCoverage=ANGLE_COVERAGE,
                            AngShift=ANGLE_SHIFT,
                            Angles=None, #np.load(f"{RESULTS_DIR}/custom_true_angles.npy"), #np.load(f"{RESULTS_DIR}/custom_true_angles.npy"),
                            angles_gen_mode="uniform_angles",#"uniform_angles",
                            output_file_name=f"{PROTEIN}_ProjectionsAngles_ProjNber{PROJECTIONS_NUM_SINGLE}_AngCoverage{','.join(map(str, ANGLE_COVERAGE))}_AngShift{','.join(map(str, ANGLE_COVERAGE))}.h5",  
                            dtype=DTYPE)

* Generating the dataset *
Protein:         5j0n
Input filename:  ../../data/5j0n.mrc
Output filename: 5j0n_ProjectionsAngles_ProjNber2000_AngCoverage2.0,1.0,2.0_AngShift2.0,1.0,2.0.h5
Volume:          (69, 57, 75)
Projections (#): (2000, 116, 116)
Angles (#):      (2000, 3)

**********


In [4]:
import numpy as np
from os import mkdir
from os.path import join, isdir
from imageio import imread, imwrite
from cryoem.rotation_matrices import RotationMatrix
import astra
import mrcfile
from pathlib import Path

def reconstruct(projections, angles, vol_shape, batch_size=50, mrc_filename=None, overwrite=False):
    reconstruction = 0
    # Generate projs with ASTRA by batches 
#     Iter = int(len(projections_all)/batch_size) 
#     for i in range(Iter):
#         projections = projections_all[i*batch_size : (i + 1)*batch_size, :, :]
#         angles = angles_all[i*batch_size : (i + 1)*batch_size, :]
        
    # Generate orientation vectors based on angles
    orientation_vectors   = RotationMatrix(angles)

    # Reshape projections correctly 
    projections1 = np.transpose(projections, (1, 0, 2))

    # Get projection dimension
    proj_size = projections1.shape[0]
    if vol_shape is None:
        vol_shape = [proj_size, proj_size, proj_size]

    # Create projection 2D geometry in ASTRA
    proj_geom = astra.create_proj_geom('parallel3d_vec', proj_size, proj_size, orientation_vectors)
    projections_id = astra.data3d.create('-sino', proj_geom, projections1)

    # Create reconstruction.
    vol_geom = astra.creators.create_vol_geom(vol_shape[1], vol_shape[2], vol_shape[0])
    reconstruction_id = astra.data3d.create('-vol', vol_geom, data=reconstruction)

    alg_cfg = astra.astra_dict('BP3D_CUDA')
    alg_cfg['ProjectionDataId'] = projections_id
    alg_cfg['ReconstructionDataId'] = reconstruction_id
    algorithm_id = astra.algorithm.create(alg_cfg)
    astra.algorithm.run(algorithm_id)
    reconstruction = astra.data3d.get(reconstruction_id)


    # Limit and scale reconstruction.
#     reconstruction[reconstruction < 0] = 0
#     reconstruction /= np.max(reconstruction)
#     reconstruction = np.round(reconstruction * 255).astype(np.uint8)

    # Cleanup.
    astra.algorithm.delete(algorithm_id)
    astra.data3d.delete(reconstruction_id)
    astra.data3d.delete(projections_id)


    # Save reconstruction to mrc file for chimera
    if mrc_filename:
        Path(mrc_filename).parent.mkdir(parents=True, exist_ok=True)
        with mrcfile.new(mrc_filename, overwrite=overwrite) as mrc:
            mrc.set_data(reconstruction)
#             mrc.header.nx=75
#             mrc.header.nx=57
#             mrc.header.nx=69
            mrc.header.cella = (275, 209, 253)
            mrc.header.cellb = (90,90,90)
            mrc.header.origin= (69.753, 63.1, 26.546001)
            mrc.header.ispg=0
            #mrc.header.mode=2
        
    return reconstruction

# def reconstruct_from_file(input_file, limit=3000, mrc_filename=None):
#     data = np.load(f'data/{input_file}.npz')
#     projections, angles = data["arr_0"].astype(np.float64)[:limit, :, :], data["arr_1"].astype(np.float64)[:limit, :]

#     return reconstruct(projections, angles, mrc_filename)


In [5]:
import h5py

In [6]:
# half coverage (AngCoverage=0.5)
projections_filename = f"{PROTEIN}_ProjectionsAngles_ProjNber{PROJECTIONS_NUM_SINGLE}_AngCoverage{','.join(map(str, ANGLE_COVERAGE))}_AngShift{','.join(map(str, ANGLE_COVERAGE))}.h5"

# load structures
data = h5py.File(projections_filename, 'r')

projections = data["Projections"]
angles_true = data["Angles"]

In [7]:
projections.shape

(2000, 116, 116)

In [8]:
type(projections[0][0][0])

numpy.float32

In [9]:
reconstruction = reconstruct(projections, angles_true, mrc_filename="5j0n_2k_new.mrc", overwrite=True, vol_shape=[69, 57, 75])


In [11]:
mlab.contour3d(reconstruction, transparent=True)
mlab.colorbar(orientation='vertical')
# mlab.savefig("data/5a1a_projections_and_angles.tiff")
mlab.show()